# ЗАДАЧА

Dota 2 — многопользовательская компьютерная игра жанра MOBA. Игроки играют между собой матчи. В каждом матче, как правило, участвует 10 человек. Матчи формируются из живой очереди, с учётом уровня игры всех игроков. Перед началом игры игроки автоматически разделяются на две команды по пять человек. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника, трон.

Вам нужно построить модель, которая по данным о первых пяти минутах матча будет предсказывать его исход — то есть определять команду-победителя.

Чтобы выполнить это задание, вам необходимо провести ряд исследований, сравнить несколько алгоритмов машинного обучения и проверить эффект от ряда манипуляций с признаками. Также, если вам понравится работать с этими данными, вы можете принять участие в соревновании на Kaggle и сравнить свои навыки с другими участниками курса!

К заданию приложены следующие файлы:

final-statement.ipynb и final-statement.html — постановка задачи, описание данных, инструкции по выполнению
features.zip — архив с обучающей выборкой
features_test.zip — архив с тестовой выборкой
data.zip — полный архив с сырыми данными и скриптом для извлечения признаков (этот архив понадобится вам только для участия в kaggle; для выполнения данного задания он не нужен)
extract_features.py — скрипт, извлекающий признаки из сырых данных

In [18]:
import pandas as pd
import numpy as np
import math
import sklearn

import re
import scipy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import r2_score, make_scorer
from sklearn.preprocessing import StandardScaler

In [171]:
features = pd.read_csv('./features.csv', index_col='match_id', header = 0)
features_test = pd.read_csv('./features_test.csv', index_col='match_id', header = 0)

Уменьшение размера выборки (ввиду низкой скорости работы):

In [172]:
features = features.sample(n=int(len(features)/2))

In [239]:
X = features.iloc[:, 0:102]
X

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
43939,1444079156,1,106,4,1205,868,9,0,0,9,...,2,-45.0,NaN,-83.0,186.0,1,4,3,3,-36.0
72776,1447893839,7,74,5,1893,1158,15,0,0,5,...,0,-16.0,NaN,-70.0,NaN,1,3,2,0,-16.0
67687,1447313398,1,71,4,1452,1706,8,2,0,8,...,1,-53.0,221.0,NaN,259.0,7,2,4,1,-30.0
21248,1438308208,1,46,5,2130,1335,15,0,0,4,...,1,-1.0,123.0,-87.0,294.0,2,3,2,1,-8.0
8406,1433665777,0,6,4,1693,1541,19,1,0,12,...,0,55.0,NaN,-86.0,NaN,6,4,2,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63129,1447002951,1,28,4,1522,1242,18,0,1,10,...,0,-37.0,190.0,-87.0,225.0,2,4,2,1,-7.0
71999,1447771790,7,7,2,780,973,1,1,0,7,...,1,-28.0,NaN,-87.0,183.0,3,3,3,0,-45.0
73916,1448080706,7,22,4,1200,1316,10,1,1,10,...,0,12.0,244.0,-83.0,NaN,1,4,2,0,205.0


# Градиентный бустинг

In [245]:
res = [X.count()[i] for i in range(len(X.count()))]

In [246]:
indexes = []
for i in range (len(res)):
    if (res[i] < 2*24307):
        indexes.append(i)

Признаки с пропусками: first_blood_time first_blood_team first_blood_player1 first_blood_player2 radiant_bottle_time radiant_courier_time radiant_flying_courier_time radiant_first_ward_time dire_bottle_time dire_courier_time dire_flying_courier_time dire_first_ward_time

In [248]:
features.iloc[1][indexes]

first_blood_time               244.0
first_blood_team                 0.0
first_blood_player1              4.0
first_blood_player2              9.0
radiant_bottle_time            167.0
radiant_courier_time           -60.0
radiant_flying_courier_time    261.0
radiant_first_ward_time        -16.0
dire_bottle_time                 NaN
dire_courier_time              -70.0
dire_flying_courier_time         NaN
dire_first_ward_time           -16.0
Name: 72776, dtype: float64

Интерпретация:

dire_flying_courier_time - апгрейд курьера команды Dire происходит после пятой минуты.  
dire_first_ward_time - первый вард команды Dire устанавливался после пятой минуты.

In [177]:
X = X.fillna(0).to_numpy()

Название столбца с целевой переменной: radiant_win.

In [178]:
Y = features.iloc[:, 103:104].to_numpy().ravel()

In [179]:
roc_auc_scorer = make_scorer(roc_auc_score)

In [129]:
grid = {'n_estimators': [10, 20, 30, 40, 50, 100, 150, 250]}
cv = KFold(n_splits=5, shuffle=True, random_state=211)
clf = GradientBoostingClassifier(verbose=True, random_state=211)
gs = GridSearchCV(clf, grid, scoring= roc_auc_scorer, cv=cv)
res = gs.fit(X, Y)

      Iter       Train Loss   Remaining Time 
         1           1.3777            4.28s
         2           1.3718            3.83s
         3           1.3655            2.97s
         4           1.3604            2.40s
         5           1.3551            1.91s
         6           1.3497            1.47s
         7           1.3444            1.08s
         8           1.3397            0.75s
         9           1.3349            0.40s
        10           1.3307            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3775            2.68s
         2           1.3716            2.40s
         3           1.3658            2.09s
         4           1.3607            1.79s
         5           1.3552            1.50s
         6           1.3499            1.19s
         7           1.3453            0.90s
         8           1.3411            0.60s
         9           1.3363            0.30s
        10           1.3320            0.00s
      It

         2           1.3718           15.14s
         3           1.3655           18.65s
         4           1.3604           17.51s
         5           1.3551           15.69s
         6           1.3497           14.65s
         7           1.3444           13.57s
         8           1.3397           13.11s
         9           1.3349           12.84s
        10           1.3307           12.48s
        20           1.2951            7.15s
        30           1.2703            3.49s
        40           1.2519            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3775           11.19s
         2           1.3716           14.12s
         3           1.3658           16.82s
         4           1.3607           17.30s
         5           1.3552           15.54s
         6           1.3499           14.26s
         7           1.3453           13.23s
         8           1.3411           12.40s
         9           1.3363           11.68s
        1

       100           1.1912            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3785           36.53s
         2           1.3724           41.21s
         3           1.3661           41.97s
         4           1.3612           39.82s
         5           1.3563           37.05s
         6           1.3510           35.19s
         7           1.3460           34.25s
         8           1.3416           33.12s
         9           1.3371           32.19s
        10           1.3329           31.27s
        20           1.2997           30.66s
        30           1.2760           26.94s
        40           1.2582           22.74s
        50           1.2427           19.25s
        60           1.2303           15.30s
        70           1.2197           11.39s
        80           1.2100            7.59s
        90           1.2015            3.77s
       100           1.1935            0.00s
      Iter       Train Loss   Remaining Time 
        

      Iter       Train Loss   Remaining Time 
         1           1.3775            1.23m
         2           1.3716            1.22m
         3           1.3658            1.34m
         4           1.3607            1.43m
         5           1.3552            1.50m
         6           1.3499            1.49m
         7           1.3453            1.46m
         8           1.3411            1.43m
         9           1.3363            1.40m
        10           1.3320            1.37m
        20           1.2978            1.27m
        30           1.2732            1.20m
        40           1.2552            1.14m
        50           1.2404            1.07m
        60           1.2277            1.02m
        70           1.2167           57.52s
        80           1.2071           56.21s
        90           1.1989           54.26s
       100           1.1912           52.07s
       200           1.1325           17.28s
      Iter       Train Loss   Remaining Time 
        

In [130]:
res.cv_results_

{'mean_fit_time': array([ 3.524018  ,  6.74609218, 11.34714427, 13.69673181, 16.95702715,
        35.33662353, 50.20706058, 83.06706228]),
 'std_fit_time': array([0.38439028, 0.53193641, 0.53564724, 0.2417217 , 0.57888267,
        2.51352613, 0.70395865, 1.71757404]),
 'mean_score_time': array([0.00880194, 0.00820894, 0.014607  , 0.01280313, 0.01321445,
        0.0210043 , 0.03320336, 0.03821344]),
 'std_score_time': array([0.0011661 , 0.0003928 , 0.00426474, 0.00312237, 0.00193704,
        0.00501707, 0.01690163, 0.00597266]),
 'param_n_estimators': masked_array(data=[10, 20, 30, 40, 50, 100, 150, 250],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 10},
  {'n_estimators': 20},
  {'n_estimators': 30},
  {'n_estimators': 40},
  {'n_estimators': 50},
  {'n_estimators': 100},
  {'n_estimators': 150},
  {'n_estimators': 250}],
 'split0_test_score': array([0.6004828 , 0.61561631, 

Кросс-валидация для градиентного бустинга с 30 деревьями проводилась 11.34714427 секунд.

Получено качество 0.62916468.

Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Да, имеет (см. 'mean_test_score').  

Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев? Убрать категориальные признаки из обучающей выборки. 

# Логистическая регрессия

## I итерация

In [180]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [181]:
grid = {'C': np.power(10.0, np.arange(-10, -2))}
cv = KFold(n_splits=5, shuffle=True, random_state=211)
clf = LogisticRegression(penalty = 'l2', random_state=211)
gs = GridSearchCV(clf, grid, scoring=roc_auc_scorer, cv=cv)
res = gs.fit(X_scaled, Y)

In [182]:
res.cv_results_

{'mean_fit_time': array([0.29291062, 0.21100941, 0.22460461, 0.22980294, 0.24705768,
        0.19260497, 0.20000825, 0.37280579]),
 'std_fit_time': array([0.05400453, 0.02186276, 0.02534637, 0.02940052, 0.04587625,
        0.05728236, 0.01136358, 0.09391039]),
 'mean_score_time': array([0.00719624, 0.00959215, 0.00859804, 0.0103982 , 0.00959663,
        0.01059628, 0.00778947, 0.00879731]),
 'std_score_time': array([0.00097737, 0.00224641, 0.00136034, 0.00326232, 0.00332034,
        0.00427123, 0.00411167, 0.00222904]),
 'param_C': masked_array(data=[1e-10, 1e-09, 1e-08, 1e-07, 1e-06, 1e-05, 0.0001,
                    0.001],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1e-10},
  {'C': 1e-09},
  {'C': 1e-08},
  {'C': 1e-07},
  {'C': 1e-06},
  {'C': 1e-05},
  {'C': 0.0001},
  {'C': 0.001}],
 'split0_test_score': array([0.5       , 0.5       , 0.5       , 0.5       , 0.50075594,
       

Какое качество получилось у логистической регрессии над всеми исходными признаками? 
0.65314076

Как оно соотносится с качеством градиентного бустинга? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом? 
Логистическая регрессия показала себя лучше, чем градиентный бустинг (как в случае качества, так и времени). Коэффициент ускорения k = 


In [249]:
11.34714427/0.37280579

30.437146027157997

## II итерация

In [183]:
X_filter = features.iloc[:, 0:102].drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)

In [184]:
X_filter = X_filter.fillna(0).to_numpy()

In [185]:
scaler_f = StandardScaler()
scaler_f.fit(X_filter)
X_filter_scaled = scaler_f.transform(X_filter)

In [186]:
grid = {'C': np.power(10.0, np.arange(-10, -2))}
cv = KFold(n_splits=5, shuffle=True, random_state=211)
clf = LogisticRegression(penalty = 'l2', random_state=211)
gs = GridSearchCV(clf, grid, scoring=roc_auc_scorer, cv=cv)
res = gs.fit(X_filter_scaled, Y)

In [187]:
res.best_params_

{'C': 0.001}

In [188]:
res.cv_results_

{'mean_fit_time': array([0.20540252, 0.18611193, 0.26701484, 0.18100319, 0.15060959,
        0.15000401, 0.18520627, 0.28081203]),
 'std_fit_time': array([0.0070603 , 0.00777221, 0.02360078, 0.0291452 , 0.01318321,
        0.00836195, 0.0102449 , 0.00831132]),
 'mean_score_time': array([0.00699706, 0.00839481, 0.00939898, 0.00759578, 0.00639858,
        0.00819454, 0.00779877, 0.00738988]),
 'std_score_time': array([0.00141357, 0.00196306, 0.00149805, 0.00338642, 0.00079983,
        0.00074666, 0.00116931, 0.00135258]),
 'param_C': masked_array(data=[1e-10, 1e-09, 1e-08, 1e-07, 1e-06, 1e-05, 0.0001,
                    0.001],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1e-10},
  {'C': 1e-09},
  {'C': 1e-08},
  {'C': 1e-07},
  {'C': 1e-06},
  {'C': 1e-05},
  {'C': 0.0001},
  {'C': 0.001}],
 'split0_test_score': array([0.5       , 0.5       , 0.5       , 0.5       , 0.50086393,
       

Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Качество несколько ухудшилось. 0.65307906

Коэффициент ускорения k =


In [250]:
11.34714427/0.28081203

40.40832677289502

Чем вы можете объяснить это изменение? Категориальные признаки подразумевают иную семантику, чем обычные числовые признаки (принадлежность к категории, кто бы мог подумать?) и, следовательно, их прямое взвешивание не может положительно повлиять на качество работы алгоритма. Тем не менее, существенного отрицательного эффекта также не наблюдается.   

## III итерация

In [189]:
heroes = features[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]

In [190]:
heroes = np.unique(heroes.to_numpy())
heroes

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 109, 110, 112], dtype=int64)

Сколько различных идентификаторов героев существует в данной игре? 
Исходя из данных, не менее 112.


In [194]:
N = 112

In [196]:
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [198]:
X_united = np.concatenate([X_filter, X_pick], axis = 1)

In [199]:
scaler_u = StandardScaler()
scaler_u.fit(X_united)
X_united_scaled = scaler_u.transform(X_united)

In [210]:
grid = {'C': np.power(10.0, np.arange(-10, -2))}
cv = KFold(n_splits=5, shuffle=True, random_state=211)
clf = LogisticRegression(penalty = 'l2', random_state=211)
gs = GridSearchCV(clf, grid, scoring=roc_auc_scorer, cv=cv)
res = gs.fit(X_united_scaled, Y)

In [201]:
res.best_params_

{'C': 0.001}

In [202]:
res.cv_results_

{'mean_fit_time': array([0.31040587, 0.30960407, 0.36949439, 0.23820329, 0.33280301,
        0.2846034 , 0.30080328, 0.59490261]),
 'std_fit_time': array([0.00487689, 0.03251427, 0.03014051, 0.01325739, 0.02831956,
        0.03307187, 0.00990351, 0.06824679]),
 'mean_score_time': array([0.01019449, 0.00999575, 0.01179805, 0.00799494, 0.01610322,
        0.01099706, 0.01139631, 0.01219606]),
 'std_score_time': array([0.00213703, 0.00141476, 0.002562  , 0.00063204, 0.00576097,
        0.00167602, 0.00224989, 0.00264164]),
 'param_C': masked_array(data=[1e-10, 1e-09, 1e-08, 1e-07, 1e-06, 1e-05, 0.0001,
                    0.001],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1e-10},
  {'C': 1e-09},
  {'C': 1e-08},
  {'C': 1e-07},
  {'C': 1e-06},
  {'C': 1e-05},
  {'C': 0.0001},
  {'C': 0.001}],
 'split0_test_score': array([0.5       , 0.5       , 0.5       , 0.5       , 0.50086393,
       

Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить? 0.67914643. Качество улучшилось и превзошло все предыдущие результаты ввиду корректной конвертации категориального признака.

## Получение предсказаний

In [203]:
Xt_filter = features_test.iloc[:, 0:102].drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
Xt_filter = Xt_filter.fillna(0).to_numpy()

In [206]:
Xt_pick = np.zeros((features_test.shape[0], N))

for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick[i, features_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [207]:
Xt_united = np.concatenate([Xt_filter, Xt_pick], axis = 1)

In [223]:
scaler_t = StandardScaler()
scaler_t.fit(Xt_united)
Xt_united_scaled = scaler_t.transform(Xt_united)

In [224]:
clf = LogisticRegression(penalty = 'l2', random_state=211, C = 0.001)
clf.fit(X_united_scaled, Y)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=211, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [225]:
np.set_printoptions(threshold=np.inf)

In [251]:
probs = clf.predict_proba(Xt_united_scaled)[:, 1]


Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [256]:
print(str(min(probs)) + " " + str(max(probs)))

0.00814355495445414 0.9895247240588367


In [233]:
result = pd.DataFrame(probs, index=features_test.index, columns=['radiant_win'])

In [235]:
result.to_csv('dota_result.csv')